In [10]:
import pandas as pd
import csv
import re
from random import randint
import numpy as np
from pandas import Series, DataFrame
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
with open('GooglePlaystore.xlsx') as file:
    
    # IF python gives IOPUB data rate exceeded
    # jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
    
    pd.set_option("display.max_rows", None, "display.max_columns", None)

    df = pd.read_excel('GooglePlaystore.xlsx')
    dfd = df.copy()
    string1 = '.'
    
    #remove '3.0M' outlier 
    for index, row in dfd.iterrows():
        if dfd.at[index, 'Reviews'] == '3.0M':
            dfd.loc[index, 'Reviews'] = np.nan
    
    #remove rows containing 'Varies with Device'
    for (colname, coldata) in dfd.iteritems():
        dfd = dfd.loc[dfd[colname] != "Varies with device"]

    #strip version string, convert to float
    dfd['Android Ver'] = dfd['Android Ver'].str[:3]
    dfd['Android Ver'] = dfd['Android Ver'].astype(float)
      
    #Installs Section
    dfd['Installs'] = dfd['Installs'].str.replace(',','')
    dfd['Installs'] = dfd['Installs'].str.replace('+','')
    dfd = dfd.loc[dfd['Installs'] != "Free"]
    dfd['Installs'] = dfd['Installs'].astype(int)
    
    #Ratings Section
    dfd['Reviews'] = dfd['Reviews'].astype(int)
    dfd['Rating'] = dfd['Rating'].fillna(0)
    for index1, row in dfd.iterrows():
        count = 0
        average = 0
        if dfd.at[index1, 'Rating'] == 0:
            if dfd.at[index1, 'Reviews'] < 100 and dfd.at[index1, 'Installs'] < 50000:
                dfd = dfd.drop(index1)
            else:
                category = dfd.at[index1, 'Category']
                for index2, row in dfd.iterrows():
                    if dfd.at[index2, 'Category'] == category and index2 != index1:
                        average += dfd.at[index2, 'Rating']
                        count += 1
                        
                dfd.at[index1, 'Rating'] = round((average/count), 2)
                
    #Size Section          
    dfd['Size'] = dfd['Size'].str.replace('M','000000', flags=re.IGNORECASE)
    dfd['Size'] = dfd['Size'].str.replace('K','000', flags=re.IGNORECASE)
    for index, row in dfd.iterrows():
        if string1 in dfd.at[index, 'Size']:
            dfd.at[index, 'Size'] = dfd.at[index, 'Size'][:-1]
            dfd.at[index, 'Size'] = dfd.at[index, 'Size'].replace('.','')
            
    dfd['Size'] = dfd['Size'].astype(int)
    
    #Describe
    new = dfd[['Category', 'Rating']]
    describestats = new.groupby('Category').describe().unstack(1) #printing provides answer
    
    #Dataframe of free Apps
    dffreeapps = dfd[(dfd.Type == 'Free')]

    #the top3 function - column must be 'Rating' / Installs' / Reviews
    # for some reason the original dataframe has duplicate entries - Subway Surfers App in 1654, 1700, 1750
    # so my implementation doesnt print 3 unique apps just top 3 in group as asked
    def topfreeapps(column, dffreeapps):
    
        if column == 'Rating':
            new = dffreeapps[['Category', 'App', 'Rating']].groupby(["Category"]).apply(lambda x: x.sort_values(["Rating"], ascending = False)).reset_index(drop=True)
            new = new.groupby('Category').head(3)
            return new
        
        elif column == 'Installs':
            new = dffreeapps[['Category', 'App', 'Installs']].groupby(["Category"]).apply(lambda x: x.sort_values(["Installs"], ascending = False)).reset_index(drop=True)
            new = new.groupby('Category').head(3)
            return new
        
        elif column == 'Reviews':
            new = dffreeapps[['Category', 'App', 'Reviews']].groupby(["Category"]).apply(lambda x: x.sort_values(["Reviews"], ascending = False)).reset_index(drop=True)
            new = new.groupby('Category').head(3)
            return new
            
        return None
    
    #find average / max / min PRICE of paid apps
    dfpaidapps = dfd[(dfd.Type == 'Paid')]
    dfpaidapps['Price'] = dfpaidapps['Price'].astype(float)
    avmaxmin = dfpaidapps['Price'].describe().loc[['mean', 'max', 'min']] #printing provides answer
    
    #genres
    dfd.explode('Genres')
    
    #UNKNOWN ERROR   
   # abc = dfd.loc[(dfd.Category == 'Business') and (dfd.Category == 'Education'),[dfd.Rating]]
    #dfd.plot.pie(dfd.Genres.value_counts())
    
    #UNKNOWN ERROR 
    #finalVis = pd.crosstab(abc.Category,abc.Rating)
    #finalVis.plot(kind = 'bar')



<ipython-input-10-794840a0f6e5>:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  dfd['Installs'] = dfd['Installs'].str.replace('+','')
<ipython-input-10-794840a0f6e5>:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfpaidapps['Price'] = dfpaidapps['Price'].astype(float)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().